## API Interaction - Push Dataframe to ADT Project
In this example we log into a remote teamworkcloud instance and update a model with data from a local json file.

Steps:
1. Read in the Vendor data json
2. Login to the TWC API Server
3. Retrieve a list of the workspaces (called categories in the TWC GUI)
4. Select the workspace to retrieve a list of projects
5. Retrieve a list project from the defined workspace
6. Select an ADT project to retrieve all elements (in memory json file)
7. Retrieve Literal Reals with Names
8. Match the ADT Values with the Vendor Values
9. Update the ADT Model (just block for now) 


In [1]:
# A little set up
import json
import requests # performs the curl function in python
import ipywidgets as widgets
from ipywidgets import Dropdown
from IPython.display import display
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import time
from anytree import Node, RenderTree

In [2]:
# Load the model dataframe
# Reading from a JSON array
df_input = pd.read_json('vendorA.json')
df_input

,Description,Value
0,length,3.256000
1,height,15.240000
2,width,2.635000
3,dateTime,1.000000
4,cadVolume,130.752494


In [3]:
# Lets get a list of workspaces from the teamworkcloud server
# API --> https://18.205.77.131:8111/osmc/swagger/?url=https%3A%2F%2F18.205.77.131%3A8111%2Fosmc%2Fmanual
#serverIp = '18.205.77.131' # Avian server
serverIp = '10.107.1.92' # ADT MA server
serverPort = '8111'
call = '/osmc/workspaces?includeBody=True'
# authId = "amRlaGFydDpqa2QyMjE0MDA=" # avian id
authId = "amRlaGFydDpSemhlZ3hoQ1RoWmY=" # ADT MA ID
url = f'https://{serverIp}:{serverPort}{call}'
headers={"accept":"application/ld+json","authorization":f"Basic {authId}"}
resp_ws = requests.get(url,headers=headers, verify=False) # turn of verification here since our server is not super secure
workspaces = resp_ws.json() # Convert the response content to a json format
#print(json.dumps(workspaces, indent=4))

In [4]:
# lets create a combobox to list the avalible workspaces
# Build arrays of the items
workspaceIds = {}
workspaceNames = {}

# Lets build a list of workspaces for selection
for i in range(len(workspaces["ldp:contains"])):
    workspaceIds[i] = workspaces["ldp:contains"][i][0]['@id']
    workspaceNames[i] = workspaces["ldp:contains"][i][1]["dcterms:title"]

# Fuction to monitor change of dropdown
def dropdown_eventhandler(change):
    print(change.new) # Write the selected item to the log

# Now create a dropdown list of the avalible workspaces
ws = widgets.Dropdown(options = workspaceNames.values(), description = 'Workspaces:')
ws.observe(dropdown_eventhandler, names='value')

# Displya the combobox
display(ws)

Dropdown(description='Workspaces:', options=('ADT_Projects', 'Test_Projects', 'Project_Templates', 'Uncategori…

In [5]:
# Lets match up the id from the selected workspace (poor implementation)
wsIndex = list(filter(lambda x: workspaceNames[x] == ws.value, range(len(workspaceNames))))
workspaceId = workspaceIds[wsIndex[0]]
workspaceId = 'a2bf1c2f-03fb-4cbe-bb3d-4edce160916f' # hard set to Test_Projects

In [6]:
# Now lets list the models that are withing the selected workspace and place them in a dropbox
# Ok... lets continue and list all of the the projects by project UID in this workspace
call = f'/osmc/workspaces/{workspaceId}/resources'
url = f'https://{serverIp}:{serverPort}{call}'
resp_projects = requests.get(url,headers=headers, verify=False) # turn of verification here since our server is not super secure
projectsList = resp_projects.json()
projectsUidList = projectsList[1]['kerml:resources'] # Let just extract the UIDs for each project
projectsUidList

[{'@id': '4eb3de65-53f4-4d59-aca2-62de8351e81b'},
 {'@id': '050dbf5b-f690-4494-81ee-18740b88d86c'},
 {'@id': 'ef9e7f30-2fc1-4677-a170-ec7035ed6b0e'},
 {'@id': 'b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4'},
 {'@id': '5cb74688-9804-44b1-badb-4f09c5cb861e'}]

## So you can see we show the ADT model uuid

In [7]:
# Lets loop throug the projects and create a dictionary of the resource (or model) details
projectsData = {}
for i in range(len(projectsUidList)):
    resourceId = projectsUidList[i]['@id'] # select the values for each id in the projectList
    call = f'/osmc/workspaces/{workspaceId}/resources/{resourceId}'
    url = f'https://{serverIp}:{serverPort}{call}'
    resp_projects = requests.get(url,headers=headers, verify=False) # turn of verification here since our server is not super secure
    #projectsData = json.dumps(resp_projects.json(), indent=4)
    projectsData[i] = resp_projects.json()
#print(json.dumps(projectsData, indent=4))

In [8]:
# lets create a combobox to list the avalible projects (models) in this workspace
# Build arrays of the items
projectIds = {}
projectNames = {}

# Lets build a list of workspaces for selection
for i in range(len(projectsData)):
    projectIds[i] = projectsData[i]['@base'].split("/")[7]
    projectNames[i] = projectsData[i]['metadata']['name'].split(".")[0]

# Now create a dropdown list of the avalible projects
prj = widgets.Dropdown(options = projectNames.values(), description = 'Projects:')
prj.observe(dropdown_eventhandler, names='value')

# Displya the combobox
display(prj)

Dropdown(description='Projects:', options=('TestVerdictKind', 'ADT_Model2', 'MacroEngine_Profile', 'ADT_Model'…

In [9]:
# Lets match up the id from the selected workspace (poor implementation)
prjIndex = list(filter(lambda x: projectNames[x] == prj.value, range(len(projectNames))))
projectId = projectIds[prjIndex[0]]
projectId = 'b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4' # hard set ADT_Model

In [10]:
# Ok so here is the wierd part... there is not direct way to extract the elements of a project (or model)
# So we have to perform a 'diff' between the elements of the initial commit and the latest revision. 

# So lets get the latest revision number (or max revision number)
# curl -X GET "https://18.205.77.131:8111/osmc/workspaces/bb95d8f4-fae4-490c-b764-4f83e3bba4f5/resources/272e28f2-45b7-45cb-a016-800ba747e716/revisions" -H "accept: application/json"
call = f'/osmc/workspaces/{workspaceId}/resources/{projectId}/revisions'
url = f'https://{serverIp}:{serverPort}{call}'
resp_revList = requests.get(url,headers=headers, verify=False) # turn of verification here since our server is not super secure
revisionList = resp_revList.json()
latestRevision = max(revisionList)
latestRevision

128

In [11]:
# Then to get the diff use the following to compair revision 1 to revision max... this is a little wierd... but it works
# I tought there would be a 'get' for all elements of a resource??? No... we must ask for the differential between the initial resource and its current version???
# curl -X GET "https://18.205.77.131:8111/osmc/workspaces/bb95d8f4-fae4-490c-b764-4f83e3bba4f5/resources/272e28f2-45b7-45cb-a016-800ba747e716/revisiondiff?source=1&target=44" -H "accept: application/json"
sourceRevision = 1
targetRevision = latestRevision
call = f'/osmc/workspaces/{workspaceId}/resources/{projectId}/revisiondiff?source={sourceRevision}&target={targetRevision}'
url = f'https://{serverIp}:{serverPort}{call}'
resp_elementList = requests.get(url,headers=headers, verify=False) # turn of verification here since our server is not super secure
elementList_json = resp_elementList.json()['added'] # just get the added (availibe items are removed, added, changed, and empty)
elementList = json.dumps(elementList_json) # push to flat string
elementList = elementList.replace('"','').replace("[","").replace("]","").replace(" ","") # remove the sting junk
#elementList_json
#elementList

In [12]:
# OK great.. now we have a list of elements from the selected model
# Lets no loop through these elements and build a json file of each elements specific information - This time we post :)
# curl -X POST "https://18.205.77.131:8111/osmc/resources/272e28f2-45b7-45cb-a016-800ba747e716/elements" -H "accept: application/ld+json" -H "Content-Type: text/plain" -d "aa4bdacf-c246-4865-bf50-cc9be2a16f16, 8a2153fc-ba11-4f9c-a2c0-2cb5114f2356 "
call = f'/osmc/resources/{projectId}/elements'
url = f'https://{serverIp}:{serverPort}{call}'
headers={"accept":"application/ld+json", "Content-Type":"text/plain", "authorization":f"Basic {authId}"}
resp_elementListData = requests.post(url,headers=headers, verify=False, data = elementList) # turn of verification here since our server is not super secure
elementListData = resp_elementListData.json() # just get the added (availibe items are removed, added, changed, and empty)
#elementListData

In [13]:
# Lets loop throug the selected projects elemetns and find the index of all literal real values
literalRealIndex = {}
for i in range(len(elementList_json)): # Where i is the uuid of the element in this case
    if elementListData[elementList_json[i]]['data'][0]['@type'] == ['ldp:DirectContainer', 'uml:LiteralReal']:
        literalRealIndex[i] = i # Add any key to the index that is a literal real
literalRealIndex
#elementList_json[4]
#elementListData['554e62c4-5bd6-46d5-97cc-900c35c46913']

{429: 429,
 543: 543,
 762: 762,
 905: 905,
 949: 949,
 969: 969,
 975: 975,
 1070: 1070,
 1085: 1085,
 1088: 1088,
 1329: 1329,
 1348: 1348,
 1724: 1724,
 1734: 1734,
 1742: 1742,
 1744: 1744,
 2011: 2011,
 2069: 2069,
 2267: 2267,
 2520: 2520,
 2588: 2588,
 2765: 2765,
 2811: 2811,
 2812: 2812,
 2833: 2833,
 2948: 2948,
 3057: 3057,
 3063: 3063,
 3105: 3105,
 3195: 3195,
 3215: 3215,
 3282: 3282,
 3288: 3288}

In [14]:
# So this is getting invloved and about here is where recursion starts to rear its ugly head
# Lets just get through this for now and we can build a better mouse trap the next round

valueCells = {}
values = {}
elementIds = {}

#print("Here we can modkify the vendor model values if we like.")

# We can loop through the matched elements and the build widgets to edit them
for keys in literalRealIndex:
    try:
        owenersId = elementListData[elementList_json[keys]]['data'][1]['kerml:owner']['@id']
        ownersName = elementListData[owenersId]['data'][1]['kerml:name']
        currentValue = elementListData[elementList_json[keys]]['data'][1]["kerml:esiData"]["value"]
        elementIds[keys] = elementList_json[keys]
        values[keys] = (ownersName, currentValue)
        valueCells[keys] = widgets.Text(description=ownersName, value=currentValue); #display(valueCells[keys])
        #print(elementListData[elementList_json[keys]]['data'][1]['kerml:owner']['@id'], elementListData[elementList_json[keys]]['data'][1]["kerml:esiData"]["value"])    
    except:
        pass
valueCells

{543: Text(value='3.256', description='adt_length'),
 1085: Text(value='15.24', description='adt_height'),
 1088: Text(value='0.0', description='calculatedVolume'),
 1348: Text(value='130.7524944', description='adt_cadVolume'),
 2267: Text(value='2.635', description='adt_width')}

In [15]:
# This functuon is executed after pushing the button
def updateSystemModelVals():
    for cells in valueCells:
        # Create the payload
        dataValue = {"kerml:esiData":{"value":"0.0"}} # Build the data payload
        dataValue["kerml:esiData"]["value"] = valueCells[cells].value # Update the json string
        
        # Now build the api call
        call = f'/osmc/resources/{projectId}/elements/{elementIds[cells]}'
        url = f'https://{serverIp}:{serverPort}{call}'
        headers={"accept":"application/ld+json", "authorization":f"Basic {authId}", "Content-Type":"application/ld+json"}
        
        # Have to add a new header of content type
        resp_value = requests.patch(url, headers = headers, verify = False, json = dataValue) # turn of verification here since our server is not super secure
        print(url)
        resp_value.status_code
        # print(dataValue)
        print(resp_value.content)
        
# Update the values in valueCells based on df_input
for key, widget in valueCells.items():
    # Extract the part of the description following 'adt_' to match with df_input
    description_suffix = widget.description[4:]  # Remove the 'adt_' prefix
    # Find the row in df_input where the description matches description_suffix
    match = df_input[df_input['Description'] == description_suffix]
    #print(match)
    if not match.empty:
        # Update the widget's value with the corresponding value from df_input
        new_value = str(float(match['Value'].iloc[0]))  # Convert to float then to string
        widget.value = new_value

# Display the updated widgets
for widget in valueCells.values():
    display(widget)
    
# Create a button to push for fun :)
btn = widgets.Button(description = "Update Model Values")
display(btn)
        
# The button callback
#btn.on_click(updateSystemModelVals)
updateSystemModelVals()

Text(value='3.256', description='adt_length')

Text(value='15.24', description='adt_height')

Text(value='0.0', description='calculatedVolume')

Text(value='130.7524944', description='adt_cadVolume')

Text(value='2.635', description='adt_width')

...

https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/elements/7d9e6305-3cf8-4dd8-9505-aa210a03c05c
b'[{"ldp:membershipResource":{"@id":"#7d9e6305-3cf8-4dd8-9505-aa210a03c05c"},"@type":["ldp:DirectContainer","uml:LiteralReal"],"ldp:contains":[],"ldp:hasMemberRelation":"kerml:ownedElement","@id":"","@context":"https://10.107.1.92:8111/osmc/schemas/umlElementContainer"},{"kerml:name":"","@base":"https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/elements","kerml:nsURI":"http://www.nomagic.com/magicdraw/UML/2.5.1","@type":"uml:LiteralReal","kerml:owner":{"@id":"5af330d6-faa3-487d-9d91-ea57dec42651"},"kerml:revision":"https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/revisions/129","@context":{"kerml":"https://10.107.1.92:8111/osmc/schema/kerml/20140325","uml:LiteralReal":"https://10.107.1.92:8111/osmc/schema/uml/2014345/LiteralReal"},"kerml:ownedElement":[],"kerml:modifiedTime":"20240514192946UTC","kerml:esiData":

https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/elements/5b86479d-b2db-4182-b116-d82095ba2617
b'[{"ldp:membershipResource":{"@id":"#5b86479d-b2db-4182-b116-d82095ba2617"},"@type":["ldp:DirectContainer","uml:LiteralReal"],"ldp:contains":[],"ldp:hasMemberRelation":"kerml:ownedElement","@id":"","@context":"https://10.107.1.92:8111/osmc/schemas/umlElementContainer"},{"kerml:name":"","@base":"https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/elements","kerml:nsURI":"http://www.nomagic.com/magicdraw/UML/2.5.1","@type":"uml:LiteralReal","kerml:owner":{"@id":"35ea390c-9bc4-42d2-addc-c49740d84ef6"},"kerml:revision":"https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/revisions/130","@context":{"kerml":"https://10.107.1.92:8111/osmc/schema/kerml/20140325","uml:LiteralReal":"https://10.107.1.92:8111/osmc/schema/uml/2014345/LiteralReal"},"kerml:ownedElement":[],"kerml:modifiedTime":"20240514192946UTC","kerml:esiData":

https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/elements/5ebd395f-7ad7-4abb-bb78-187550df9315
b'[{"ldp:membershipResource":{"@id":"#5ebd395f-7ad7-4abb-bb78-187550df9315"},"@type":["ldp:DirectContainer","uml:LiteralReal"],"ldp:contains":[],"ldp:hasMemberRelation":"kerml:ownedElement","@id":"","@context":"https://10.107.1.92:8111/osmc/schemas/umlElementContainer"},{"kerml:name":"","@base":"https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/elements","kerml:nsURI":"http://www.nomagic.com/magicdraw/UML/2.5.1","@type":"uml:LiteralReal","kerml:owner":{"@id":"2312a234-0675-47ab-9cdf-e1d157959fc2"},"kerml:revision":"https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/revisions/131","@context":{"kerml":"https://10.107.1.92:8111/osmc/schema/kerml/20140325","uml:LiteralReal":"https://10.107.1.92:8111/osmc/schema/uml/2014345/LiteralReal"},"kerml:ownedElement":[],"kerml:modifiedTime":"20240514192948UTC","kerml:esiData":

https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/elements/de83bcbd-7cd6-4c3e-a637-1c2dcaa3a283
b'[{"ldp:membershipResource":{"@id":"#de83bcbd-7cd6-4c3e-a637-1c2dcaa3a283"},"@type":["ldp:DirectContainer","uml:LiteralReal"],"ldp:contains":[],"ldp:hasMemberRelation":"kerml:ownedElement","@id":"","@context":"https://10.107.1.92:8111/osmc/schemas/umlElementContainer"},{"kerml:name":"","@base":"https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/elements","kerml:nsURI":"http://www.nomagic.com/magicdraw/UML/2.5.1","@type":"uml:LiteralReal","kerml:owner":{"@id":"e3b666f7-7484-4bce-b35b-5dc783b9c095"},"kerml:revision":"https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/revisions/132","@context":{"kerml":"https://10.107.1.92:8111/osmc/schema/kerml/20140325","uml:LiteralReal":"https://10.107.1.92:8111/osmc/schema/uml/2014345/LiteralReal"},"kerml:ownedElement":[],"kerml:modifiedTime":"20240514192949UTC","kerml:esiData":

https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/elements/84f3b08d-89d5-4da0-b6d8-734cf9fcb54a
b'[{"ldp:membershipResource":{"@id":"#84f3b08d-89d5-4da0-b6d8-734cf9fcb54a"},"@type":["ldp:DirectContainer","uml:LiteralReal"],"ldp:contains":[],"ldp:hasMemberRelation":"kerml:ownedElement","@id":"","@context":"https://10.107.1.92:8111/osmc/schemas/umlElementContainer"},{"kerml:name":"","@base":"https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/elements","kerml:nsURI":"http://www.nomagic.com/magicdraw/UML/2.5.1","@type":"uml:LiteralReal","kerml:owner":{"@id":"867db4f1-5548-44bf-92a6-b07d813dcd62"},"kerml:revision":"https://10.107.1.92:8111/osmc/resources/b5b69f4d-e1d0-4ddc-b514-d54b05d0a3b4/revisions/133","@context":{"kerml":"https://10.107.1.92:8111/osmc/schema/kerml/20140325","uml:LiteralReal":"https://10.107.1.92:8111/osmc/schema/uml/2014345/LiteralReal"},"kerml:ownedElement":[],"kerml:modifiedTime":"20240514192950UTC","kerml:esiData":

# I cant force the update of the block volume attribute at this time.

A couple of notes:
1. This can fully automated and you should be able to see that we can use complex methods in between the pull and push
2. I cant execute the volume update with this setup (we dont want to update blocks but instances...)
3. Matching the ADT value to the Vendor value can be performed by looking up an ICD 
3. I'll build out an instance updater tommorow :)